## Problem set 6

## Name: [Yawen Tan]

## Link to your PS6 github repo: [https://github.com/IsabellaTan/Brown-DATA1030-HW6]

### Problem 0 

-2 points for every missing green OK sign. 

Make sure you are in the DATA1030 environment.

In [1]:
from __future__ import print_function
from packaging.version import parse as Version
from platform import python_version

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.12.10 is required,"
                " but %s is installed." % sys.version)

def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == Version(min_ver):
            print(OK, "%s version %s is installed."
                  % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                  % (lib, min_ver, ver))    
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod


# first check the python version
pyversion = Version(python_version())

if pyversion >= Version("3.12.10"):
    print(OK, "Python version is %s" % pyversion)
elif pyversion < Version("3.12.10"):
    print(FAIL, "Python version 3.12.10 is required,"
                " but %s is installed." % pyversion)
else:
    print(FAIL, "Unknown Python version: %s" % pyversion)

    
print()
requirements = {'numpy': "2.2.5", 'matplotlib': "3.10.1",'sklearn': "1.6.1", 
                'pandas': "2.2.3",'xgboost': "3.0.0", 'shap': "0.47.2", 
                'polars': "1.27.1", 'seaborn': "0.13.2"}

# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.12.10

[ OK ] numpy version 2.2.5 is installed.
[ OK ] matplotlib version 3.10.1 is installed.
[ OK ] sklearn version 1.6.1 is installed.
[ OK ] pandas version 2.2.3 is installed.
[ OK ] xgboost version 3.0.0 is installed.
[ OK ] shap version 0.47.2 is installed.
[ OK ] polars version 1.27.1 is installed.
[ OK ] seaborn version 0.13.2 is installed.


## Problem 1 (5 points)

Write a function called `linear_ML_pipeline` which takes training, validation, test sets, and a boolean variable called `is_classif` as input. The variable `is_classif` is `True` if the target variable is categorial (classification problem), and `False` if the target variable is continuous (regression problem). The function also takes the following lists as inputs:
- continuous_ftrs: the column names of continuous features
- ordinal_ftrs: the column names of ordinal features
- ordinal_cats: the ordered list of categories for each ordinal feature
- categorical_ftrs: the column names of categorical features

Within the function, perform the following steps:
- write the docstring
- test the inputs! write at least 10 tests. make sure that among other things, all features are accounted for in the lists.
- preprocess the sets using sklearn and make sure to fit_transform the training set, and transform the validation and test sets
- fit a logistic regression model if `is_classif` is `True`, a linear regression model otherwise
- use the elastic net regularization and tune both hyperparameters (alpha or C and l1_ratio)
- perform CV and select the hyperparameter combo which optimizes the validation score
- calculate the test score
- your function should return the best model, its hyperparameters, and the test score

You will use this function to solve problems 2 and 3. Data splitting will be performed before the function is called.

In [3]:
# add your code here

def linear_ML_pipeline(X_train, y_train, X_val, y_val, X_test, y_test, is_classif, continuous_ftrs, ordinal_ftrs, ordinal_cats, categorical_ftrs, random_state=42):

    '''
    Build and evaluate a complete linear-model pipeline that supports both
    classification (Logistic Regression) and regression (Elastic Net),
    with preprocessing, elastic-net regularization, manual hyperparameter tuning
    over a grid, validation-based model selection, and final test evaluation.

    Parameters:
    -----------
    X_train : pandas.DataFrame
        Feature matrix for the training split. Columns must match X_val and X_test
        exactly (same names and order).
    y_train : pandas.Series or 1D array-like
        Target vector for the training split.
    X_val : pandas.DataFrame
        Feature matrix for the validation split (used to select hyperparameters).
    y_val : pandas.Series or 1D array-like
        Target vector for the validation split.
    X_test : pandas.DataFrame
        Feature matrix for the test split (held out for final evaluation only).
    y_test : pandas.Series or 1D array-like
        Target vector for the test split.
    is_classif : bool
        If True, fit a Logistic Regression model (elastic net penalty) for a
        classification task; if False, fit an ElasticNet regressor for a
        regression task.
    continuous_ftrs : list[str]
        Column names (in X_*) that are continuous features to be standardized
        (mean=0, std=1).
    ordinal_ftrs : list[str]
        Column names (in X_*) that are ordinal categorical features to be
        encoded via OrdinalEncoder using the specified order in `ordinal_cats`.
    ordinal_cats : list[list[Any]]
        Ordered categories for each ordinal feature. The i-th list provides the
        category order for ordinal_ftrs[i].
    categorical_ftrs : list[str]
        Column names (in X_*) that are nominal (unordered) categorical features
        to be encoded via OneHotEncoder(handle_unknown='ignore').
    random_state : int, default=42
        Random seed used where applicable (e.g., solvers), to ensure reproducibility.

    Returns:
    --------
    best_model : sklearn estimator (already fitted)
        The model refit on TRAIN+VAL using the best hyperparameters discovered
        on the validation set (classification: LogisticRegression with penalty='elasticnet'
        and solver='saga'; regression: ElasticNet).
    best_hyperparams : dict
        The best hyperparameter combination found on the validation set. Includes
        at least {'l1_ratio': ..., 'C': ...} for classification or
        {'l1_ratio': ..., 'alpha': ...} for regression, and any other key settings
        needed to reproduce the best model.
    test_score : float
        Final performance on the held-out test set. For classification, accuracy;
        for regression, R^2. (These defaults follow the introductory course
        conventions.)

    Example:
    --------
    >>> # Suppose you already split your data and defined feature groups:
    >>> best_model, best_params, test_score = linear_ML_pipeline(
    ...     X_train, y_train, X_val, y_val, X_test, y_test,
    ...     is_classif=True,
    ...     continuous_ftrs=['age','capital_gain','capital_loss','hours_per_week'],
    ...     ordinal_ftrs=['education'],
    ...     ordinal_cats=[[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',
    ...                    ' 10th',' 11th',' 12th',' HS-grad',' Some-college',
    ...                    ' Assoc-voc',' Assoc-acdm',' Bachelors',' Masters',
    ...                    ' Prof-school',' Doctorate']],
    ...     categorical_ftrs=['workclass','marital-status','occupation',
    ...                       'relationship','race','sex','native-country'],
    ...     random_state=42
    ... )
    >>> best_params
    {'penalty': 'elasticnet', 'solver': 'saga', 'C': 1.0, 'l1_ratio': 0.5, 'max_iter': 10000}
    >>> test_score
    0.85  # accuracy on the held-out test set (example)
    '''
    # import library
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LogisticRegression, ElasticNet
    from sklearn.model_selection import ParameterGrid
    from sklearn.metrics import accuracy_score, r2_score, mean_squared_error


    # TEST
    #----------------------------------------------------------------
    # Test 1: if X_train, X_val, X_test are not pandas DataFrames, raise ValueError
    if not all(isinstance(x, pd.DataFrame) for x in [X_train, X_val, X_test]):
        raise ValueError("X_train, X_val, and X_test must all be pandas DataFrames")

    # Test 2: if y_train, y_val, y_test are not pandas Series, raise ValueError
    if not all(isinstance(y, (pd.Series, np.ndarray, list)) for y in [y_train, y_val, y_test]):
        raise ValueError("y_train, y_val, and y_test must each be a pandas Series, numpy array, or list")

    # Test 3: if any of the feature DataFrames are empty (0 rows or 0 columns), raise ValueError
    for name, df in zip(["X_train", "X_val", "X_test"], [X_train, X_val, X_test]):
        if df.shape[0] == 0:
            raise ValueError(f"{name} has 0 rows")
        if df.shape[1] == 0:
            raise ValueError(f"{name} has 0 columns")

    # Test 4: ensure column names and order match across X_train, X_val, X_test
    if list(X_train.columns) != list(X_val.columns) or list(X_train.columns) != list(X_test.columns):
        raise ValueError("Column names and order must be identical across X_train, X_val, and X_test")

    # Test 5: ensure feature lists are all Python lists
    for var_name, var in {
        "continuous_ftrs": continuous_ftrs,
        "ordinal_ftrs": ordinal_ftrs,
        "ordinal_cats": ordinal_cats,
        "categorical_ftrs": categorical_ftrs,
    }.items():
        if not isinstance(var, list):
            raise ValueError(f"{var_name} must be provided as a list")

    # Test 6: check that every feature column is accounted for exactly once
    all_features = continuous_ftrs + ordinal_ftrs + categorical_ftrs
    if set(all_features) != set(X_train.columns):
        missing = set(X_train.columns) - set(all_features)
        extra = set(all_features) - set(X_train.columns)
        raise ValueError(f"Feature mismatch: missing {missing}, extra {extra}")

    # Test 7: check for overlaps among feature groups
    if (set(continuous_ftrs) & set(ordinal_ftrs)) or \
       (set(continuous_ftrs) & set(categorical_ftrs)) or \
       (set(ordinal_ftrs) & set(categorical_ftrs)):
        raise ValueError("continuous_ftrs, ordinal_ftrs, and categorical_ftrs must not overlap")

    # Test 8: ensure ordinal_ftrs and ordinal_cats have the same length
    if len(ordinal_ftrs) != len(ordinal_cats):
        raise ValueError("Length of ordinal_cats must match the number of ordinal_ftrs")

    # Test 9: ensure continuous_ftrs is not empty
    if continuous_ftrs == [] and ordinal_ftrs == [] and categorical_ftrs == []:
        raise ValueError("At least one feature list must be non-empty")

    # Test 10: ensure is_classif is a boolean
    if not isinstance(is_classif, bool):
        raise ValueError("is_classif must be a boolean value (True for classification, False for regression)")

    # Test 11: ensure target lengths match feature lengths
    for (x, y, name) in [(X_train, y_train, "train"), (X_val, y_val, "validation"), (X_test, y_test, "test")]:
        if len(x) != len(y):
            raise ValueError(f"Length mismatch between X_{name} and y_{name}")

    # Test 12: optional—check for duplicate columns in X_train
    if X_train.columns.duplicated().any():
        raise ValueError("Duplicate column names detected in X_train")
    



    # Data Preprocessing
    #----------------------------------------------------------------
    # Continuous features: StandardScaler
    continuous_transformer = Pipeline(steps=[('scaler', StandardScaler())])

    # Ordinal features: OrdinalEncoder
    ordinal_transformer = Pipeline(steps=[('encoder', OrdinalEncoder(categories=ordinal_cats))])

    # Nominal (categorical) features: OneHotEncoder
    categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

    # Combine all three transformers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('cont', continuous_transformer, continuous_ftrs),
            ('ord', ordinal_transformer, ordinal_ftrs),
            ('cat', categorical_transformer, categorical_ftrs)],
        remainder='drop'  # if feature list cover all the column, drop left feature
        )

    # Fit on training data, transform all sets
    X_train_prep = preprocessor.fit_transform(X_train)
    X_val_prep = preprocessor.transform(X_val)
    X_test_prep = preprocessor.transform(X_test)



    #  Model setup and Elastic Net regularization
    #----------------------------------------------------------------
    from sklearn.linear_model import LogisticRegression, ElasticNet
    from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
    from sklearn.model_selection import ParameterGrid

    if is_classif:
        # Logistic Regression + Elastic Net Regularization
        model_type = "classification"
        base_model = LogisticRegression(
            penalty="elasticnet",
            solver="saga",
            max_iter=3000,
            random_state=random_state)

        # Parameter tuning
        param_grid = {
            "C": np.logspace(-4, 4, 5),       # [1e-4, 1e-2, 1, 1e2, 1e4]
            "l1_ratio": np.linspace(0.1, 1.0, 5)}
        score_func = accuracy_score
        score_name = "Accuracy"

    else:
        # Linear Regression：ElasticNet
        model_type = "regression"
        base_model = ElasticNet(
            max_iter=5000,
            random_state=random_state)

        # Parameter tuning
        param_grid = {
            "alpha": np.logspace(-4, 1, 6),   # [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
            "l1_ratio": np.linspace(0.1, 1.0, 6)}

        score_func = r2_score # use R square
        mse_func = mean_squared_error
        score_name = "R²"

    # Cross-validation, hyperparameter tuning, and model selection
    # ----------------------------------------------------------------
    best_val_score = -np.inf  # record the best score for validation set
    best_params = None
    best_model = None
    # set a list to store result
    results = []

    # Loop every combinition of parameter
    for params in ParameterGrid(param_grid):
        # Create new model for new parameter
        model = base_model.set_params(**params)
        # Fit the train data
        model.fit(X_train_prep, y_train)
        # Predit on train and validation data
        y_train_pred = model.predict(X_train_prep)
        y_val_pred = model.predict(X_val_prep)
        # Calcuate the score
        train_score = score_func(y_train, y_train_pred)
        val_score = score_func(y_val, y_val_pred)
        mse_val = None
        if not is_classif:
            mse_val = mean_squared_error(y_val, y_val_pred)
        # Store the result
        results.append({
            **params,
            "train_score": train_score,
            "val_score": val_score,
            "mse_val": mse_val
        })
        # Update best model
        if val_score > best_val_score:
            best_val_score = val_score
            best_params = params
            best_model = model

    # Refit best model on TRAIN+VAL, evaluate on TEST
    # ---------------------------------------------------------
    # Combine train and validation set
    X_trainval = np.vstack([X_train_prep, X_val_prep])
    y_trainval = np.concatenate([y_train, y_val])
    # use best parameter to train the model
    final_model = base_model.set_params(**best_params)
    final_model.fit(X_trainval, y_trainval)
    # predict on the test set and calculate the score
    y_test_pred = final_model.predict(X_test_prep)
    test_score = score_func(y_test, y_test_pred)

    # For regression: use R² as the final evaluation metric (consistent with lecture and default sklearn scoring)
    return final_model, best_params, test_score



## Problem 2: time series forecasting with VAR

You will practice multivariate time series forcasting using VAR - vector autoregression.

The `stocks_prices.csv` is in the data folder. It contains the stock prices of amazon (AMZN), microsoft (MSFT), and apple (AAPL). Here is a description of each column in the dataset:
- price ticker date: the date when the stock price was recorded - note that weekends and holidays are absent
- Close AAPL: apple stock price at closing time in USD (i.e., at the end of the trading day)
- Close AMZN: amazon stock price at closing time in USD (i.e., at the end of the trading day)
- Close MSFT: microsoft stock price at closing time in USD (i.e., at the end of the trading day)
- High AAPL: highest apple stock price during the trading day in USD
- High AMZN: highest amazon stock price during the trading day in USD
- High MSFT: highest microsoft stock price during the trading day in USD
- Low AAPL: lowest apple stock price during the trading day in USD
- Low AMZN: lowest amazon stock price during the trading day in USD
- Low MSFT: lowest microsoft stock price during the trading day in USD
- Open AAPL: apple stock price at opening time in USD (i.e., at the beginning of the trading day)
- Open AMZN: amazon stock price at opening time in USD (i.e., at the beginning of the trading day)
- Open MSFT: microsoft stock price opening time in USD (i.e., at the beginning of the trading day)
- Volume AAPL: total traded volume (buys and sells) of apple on the trading day in USD
- Volume AMZN: total traded volume (buys and sells) of amazon on the trading day in USD
- Volume MSFT: total traded volume (buys and sells) of microsoft on the trading day in USD

The goal of problem 2 is to predict the opening price of apple stocks one day ahead based on the time series observations.

### Problem 2a - feature matrix (15 points)
Perform the steps outlined in the cell below.

In [ ]:
# add your code here
# import packages


# read in the dataset and print the header


# write a function which takes the following input:
# - the dataframe, 
# - the name of the target column, 
# - a variable `p` which describes how many autoregresive past features we use (p in AR(p) of the lecture notes)
# the function should return a feature matrix X and target variable y after VAR is applied to the multivariate time series data
# make sure that the points are ordered with respect to time such that 
# the oldest observations are at the top and the most recent observation are at the bottom of the dataframe.






### Problem 2b - splitting (10 points)
We will split the dataset in a couple of different ways to study information leakage. Perform the steps outlined in the cell below.

In [ ]:
# add your code here

# split 1: create shuffled train/validation/test sets (60/20/20 ratio) and run `linear_ML_pipeline` on it. 
# print out the best hyperparameter values, and the test score
# repeat this process with 5 random states
# print out the best hyperparameter values, and the test score
# print out the mean and stdev of the 5 test scores.


# split 2: place 20% of the most recent observations in the test set, 
# then apply sklearn's TimeSeriesSplit on the rest of the data with n_splits = 5
# run `linear_ML_pipeline` on each split.
# print out the best hyperparameter values, and the test score
# print out the mean and stdev of the 5 test scores.





Discuss in this cell what you observe and answer the questions below:

- the mean and stdev of the test scores differ based on how you split the data. Explain in a few sentences why!

your answer here

- Forecasting models like this can be used for trading. I.e., if your model predicts that the opening price of the apple stock tomorrow will be higher than the closing price today, you'd put in a sell order at the end of the day. If your prediction is correct, you will make a profit in USD once your order executes at tomorrow's open. Similarly, if your model predicts that the opening price of apple stock tomorrow will be lower than the closing price today, you'd put in a buy order. If your prediction is correct, you'll buy at a low price once your order executes at tomorrow's open. This is how you'd act based on the model's prediction - buy low, sell high. Would you be willing to use your own money to deploy the models developed in split 1 and/or split 2? Why or why not?

your answer here

## Problem 3 - group structure

We will work with the [hand postures dataset](https://archive.ics.uci.edu/ml/datasets/Motion+Capture+Hand+Postures) in problem 3. Please carefully read the dataset description and perform as much EDA as you can on this dataset. The EDA is not graded but it will help you to correctly answer 3a and 3b.

This dataset has group structure: 14 users performing 5 different hand postures while wearing sensors attached to a left-handed glove. Two different ML questions can be asked using this dataset. We will explore how splitting differs for both questions in 2a and 2b.

Later on, we'll teach you how to deal with missing data. For now, simply drop all columns with any missing values (**don't do this in real life**, but for now it's fine). 

In [ ]:
# add your code below


## Problem 3a - basic splitting (5 points)

Create shuffled train/validation/test sets (60/20/20 ratio) and use `linear_ML_pipeline` to predict the class. 
Print out the best hyperparameter values, and the test score.
Repeat this process with 5 random states and report the mean and stdev of the test score.

You may receive some warnings about models failing to converge. This usually happens when C is too high (aka not enough regularization). Play around with those parameters in `linear_ML_pipeline` until you no longer see that warning.

In [ ]:
# add your code here


### Problem 3b (10 points)

How would you split the dataset if we wanted to know how accurately we can predict the hand postures of a new, previously unseen user? What's the target variable? Write down your reasoning (the usual 1-2 paragraphs are fine). Split the dataset into training, validation, and test sets, and run `linear_ML_pipeline` on it.

Add your explanation here:


In [ ]:
# add your code here


### Problem 3c (10 points)

How would you split the data if we wanted to identify a user based on their hand postures? What's the target variable? Follow the same steps as in 3b (explain your reasoning, split, and run `linear_ML_pipeline`).

Add your explanation here:
